In [19]:
# Site para extração
url = "https://lista.mercadolivre.com.br/_Container_pet-cpg-caes"

In [1]:
# Instalação de bibliotecas
!pip install requests
!pip install beautifulsoup4
!pip install selenium

In [2]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [26]:
# Configuração para usar o Selenium

edge_service = Service(r"C:\Users\fred\Documents\CursoFullStack\Web Scraping\edgedriver_win64\msedgedriver.exe")
options = webdriver.EdgeOptions()
driver = webdriver.Edge(service= edge_service, options=options)

# Iniciando o Selenium

driver.get(url)
time.sleep(5)

# Obtendo as urls de cada produto

produtos_links = []


title_wrappers = driver.find_elements(By.CLASS_NAME,"poly-component__title-wrapper")
for wrapper in title_wrappers:
    try:
        link_element = wrapper.find_element(By.TAG_NAME, "a")
        href = link_element.get_attribute("href")
        if href and "mercadolivre.com.br" in href:
            produtos_links.append(href)
    except:
        continue
next_button = driver.find_element(By.XPATH, "//li[contains(@class, 'andes-pagination__button--next')]/a")
next_page_url = next_button.get_attribute("href")
driver.get(next_page_url)  
time.sleep(10)

while True:
    title_wrappers = driver.find_elements(By.CLASS_NAME,"ui-search-item__title")
    for wrapper in title_wrappers:
        try:
            link_element = wrapper.find_element(By.TAG_NAME, "a")
            href = link_element.get_attribute("href")
            if href and "mercadolivre.com.br" in href:
                produtos_links.append(href)
        except:
            continue
    try:
        next_button = driver.find_element(By.XPATH, "//li[contains(@class, 'andes-pagination__button--next')]/a")
        next_page_url = next_button.get_attribute("href")
        driver.get(next_page_url)  
        time.sleep(10)
    except:
        break

df = pd.DataFrame(produtos_links)
df.to_csv('URLs_por_Produto.csv', index=False)

In [17]:
# Recebendo dataset com Urls únicos

df_urls = pd.read_csv('URLs_por_produto.csv')
product_links = df_urls.iloc[:, 0].tolist() 

# Tabela 1 - Informações gerais

dataset1 = {'Categoria': [],
          'Produto': [],
          'Marca': [], 
          'Preco': [],
          'Avaliacao': [],
          'Quant. Avaliacoes': [],
          'Quant. Comentários': [],
          'URL': []}

# Tabela 2 - Comentários

dataset2 = {'URL': [],
           'Comentários': []}


# Tabela 3 - Avaliação de características

dataset3 = {'Característica': [],
           'Aval.Característica': [],
           'URL': []}

# Tabela 4 - Urls que não responderam

nao_capturado = []

# Declaração de variável para intervalo de requisições

requisicoes = 0
comentario = []
caracteristica = []
avaliacao_caracteristica = []

# Código a ser aplicado em cada url única

for pag in product_links:
    try:
        req_books = requests.get(pag)
        conteudo_books = BeautifulSoup(req_books.text,'html.parser')

        # Variável para nome do produto

        produto = conteudo_books.find('h1', class_="ui-pdp-title").text

        # Confirmação da declaração da variável produto

        max_tentativas = 10
        tentativa = 0
        while tentativa < max_tentativas and produto is None:
            req_books = requests.get(pag)
            conteudo_books = BeautifulSoup(req_books.text,'html.parser')
            tentativa += 1
            produto = conteudo_books.find('h1', class_="ui-pdp-title").text
            time.sleep(2)

        # Variáveis Gerais

        preco_tag = conteudo_books.find("meta", itemprop="price")
        preco = preco_tag.get("content") if preco_tag else None
        cate_tag = conteudo_books.find_all("a", class_="andes-breadcrumb__link")
        categoria = cate_tag[-1].get("title")
        avaliacao_tag = conteudo_books.find('p', class_="ui-review-capability__rating__average ui-review-capability__rating__average--desktop")
        avaliacao = avaliacao_tag.text if avaliacao_tag else None
        quant_avalia_tag = conteudo_books.find("span", class_="ui-pdp-review__amount")
        quant_avaliacao = quant_avalia_tag.text.strip('()') if avaliacao_tag else None
        quant_comentario_tag = conteudo_books.find('span', class_="total-opinion")
        quant_comentario = conteudo_books.find('span', class_="total-opinion").text if quant_comentario_tag else None

        # Confirmação da declaração da variável marca

        max_tentativas = 10
        tentativa = 0
        marca = None
        while tentativa < max_tentativas and marca is None:
            req_books = requests.get(pag)
            conteudo_books = BeautifulSoup(req_books.text,'html.parser')
            tentativa += 1
            marca = None  
            tabelas = []
            tabelas = conteudo_books.find_all("table", class_="andes-table")

            for tabela in tabelas:
                for linha in tabela.find_all("tr"):
                    th_div = linha.find("div", class_="andes-table__header__container")
                    if th_div and "Marca" in th_div.get_text(strip=True): 
                        td = linha.find("td")  
                        if td:
                            marca = td.get_text(strip=True) 
                            break  
                if marca:
                    break  

        # Variáveis para a tabela 1

        dataset1['Categoria'].append(categoria)
        dataset1['Produto'].append(produto)
        dataset1['Preco'].append(preco)
        dataset1['Avaliacao'].append(avaliacao)
        dataset1['Quant. Avaliacoes'].append(quant_avaliacao)
        dataset1['Marca'].append(marca)
        dataset1['URL'].append(pag)
        dataset1['Quant. Comentários'].append(quant_comentario)

        # Variáveis para avaliação de características

        aval_produto = conteudo_books.findAll('tr', class_="ui-review-capability-categories__desktop--row")
        if aval_produto is not None:
            for a in aval_produto:

                # Variáveis para a tabela 3

                dataset3['Característica'].append(a.find('td', class_="").text)
                dataset3['Aval.Característica'].append(a.find('p', class_="andes-visually-hidden").text)
                dataset3['URL'].append(pag)   

        # Declaração da variável comentário

        # Verificação se há comentários

        if avaliacao is not None:
            botao = conteudo_books.find("button", {"data-testid": "see-more"})

            # Se houver comentários

            # Verificação se há necessidade de usar Selenium

            if botao:

                # Usando o Selenium

                def open_and_scrape(pag):
                    options = webdriver.EdgeOptions()
                    options.add_argument("--start-maximized")

                    driver = webdriver.Edge(options=options)
                    driver.get(pag)

                    try:
                        time.sleep(3)

                        # Simular rolagem para carregar mais comentários
                        body = driver.find_element(By.TAG_NAME, "body")
                        for _ in range(15): 
                            body.send_keys(Keys.PAGE_DOWN)
                            time.sleep(1)

                        # Clicar no botão "Ver mais"
                        try:
                            see_more_button = WebDriverWait(driver, 10).until(
                                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-testid='see-more']"))
                            )
                            see_more_button.click()
                            time.sleep(3) 
                        except:
                            print("Botão 'Ver mais' não encontrado, continuando...")

                        # Declaração da variável com url do modal

                        iframe = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.ID, "ui-pdp-iframe-reviews"))
                        )
                        iframe_url = iframe.get_attribute("src")

                        # Garantir que a URL do modal seja completa
                        if "http" not in iframe_url:
                            iframe_url = "https://www.mercadolivre.com.br" + iframe_url


                        # Abrir a URL do modal diretamente

                        driver.get(iframe_url)
                        time.sleep(5)

                        # Pressionar TAB algumas vezes para garantir que o foco está no modal

                        actions = ActionChains(driver)
                        actions.send_keys(Keys.TAB).perform()
                        time.sleep(1)


                        # Simula o pressionamento de Page Down

                        prev_scroll_top = -1  # Inicializa uma variável para armazenar a posição anterior

                        while True:
                            # Executa o comando para rolar para baixo
                            actions.send_keys(Keys.PAGE_DOWN).perform()
                            time.sleep(1)  # Aguarda um pouco para a página carregar

                            # Executa JavaScript para obter a posição do scroll
                            scroll_top = driver.execute_script("return window.pageYOffset;")
                            scroll_height = driver.execute_script("return document.documentElement.scrollHeight;")
                            client_height = driver.execute_script("return window.innerHeight;")

                            # Se a posição do scroll não mudou ou atingiu o final da página, sair do loop
                            if scroll_top == prev_scroll_top or (scroll_top + client_height) >= scroll_height:
                                break

                            prev_scroll_top = scroll_top

                        # Capturar o HTML atualizado

                        page_source = driver.page_source


                    finally:
                        driver.quit()

                    # Usar BeautifulSoup para extrair os comentários
                    soup = BeautifulSoup(page_source, "html.parser")


                    # Encontrar todas as tags <p> dos comentários
                    for comment in soup.find_all("p", class_="ui-review-capability-comments__comment__content"):
                        dataset2['Comentários'].append(comment.get_text(strip=True))
                        dataset2['URL'].append(pag)
                # Executar função

                open_and_scrape(pag)

            # Se não houver necessidade de usar o Selenium

            else:
                for comment in conteudo_books.find_all("p", class_="ui-review-capability-comments__comment__content"):
                    dataset2['Comentários'].append(comment.get_text(strip=True))
                    dataset2['URL'].append(pag)
        # Se não tem comentários

        else:
            dataset2['Comentários'].append('Sem comentários')
            dataset2['URL'].append(pag)



        # Limitador de requisições em sequência para evitar bloqueio

        requisicoes +=1
        print(requisicoes)
        if requisicoes == 100:
            print('100 requisições. Pausa de 30 minutos')
            requisicoes = 0
            time.sleep(1805)
        else:
            continue

    except Exception as e:
        nao_capturado.append(pag)
        continue  # Vai para o próximo `i` sem interromper o loop

# Transformando em DataFrame e exportando para csv

df1 = pd.DataFrame(dataset1)
df2 = pd.DataFrame(dataset2)
df3 = pd.DataFrame(dataset3)
urls_faltantes = pd.DataFrame(nao_capturado)
df1.to_csv('df1.csv', sep = ';', index=False, encoding="utf-8-sig")
df2.to_csv('df2.csv', sep = ';', index=False, encoding="utf-8-sig")
df3.to_csv('df3.csv', sep = ';', index=False, encoding="utf-8-sig")
urls_faltantes.to_csv('urls_faltantes.csv',sep = ';')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
